# Web Server Log Analysis - Python Take-Home Assessment

## Overview
This assessment involves analyzing the Calgary HTTP dataset, which contains approximately one year's worth of HTTP requests to the University of Calgary's Computer Science web server. You'll work with real-world web server log data to extract meaningful insights and demonstrate your Python data analysis skills.

## Part 1: Data Loading and Cleaning

### Instructions

* Work in the cells below - You can add as many cells as needed for data loading, cleaning, and exploration
* Import required libraries
* Implement data loading and cleaning - Create functions to download, parse, and clean the log data
* Explore the data - Understand the structure and identify any data quality issues

In [16]:
file_path = r'C:\Users\swati\Desktop\calgary_access_log'

with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
   for i, line in enumerate(file):
    if i >= 10:
        break
    print(line)

local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150

local - - [24/Oct/1994:13:41:41 -0600] "GET 1.gif HTTP/1.0" 200 1210

local - - [24/Oct/1994:13:43:13 -0600] "GET index.html HTTP/1.0" 200 3185

local - - [24/Oct/1994:13:43:14 -0600] "GET 2.gif HTTP/1.0" 200 2555

local - - [24/Oct/1994:13:43:15 -0600] "GET 3.gif HTTP/1.0" 200 36403

local - - [24/Oct/1994:13:43:17 -0600] "GET 4.gif HTTP/1.0" 200 441

local - - [24/Oct/1994:13:46:45 -0600] "GET index.html HTTP/1.0" 200 3185

local - - [24/Oct/1994:13:46:45 -0600] "GET 2.gif HTTP/1.0" 200 2555

local - - [24/Oct/1994:13:46:47 -0600] "GET 3.gif HTTP/1.0" 200 36403

local - - [24/Oct/1994:13:46:50 -0600] "GET 4.gif HTTP/1.0" 200 441



In [17]:
# You can write your code here for data loading, cleaning, and exploration. Add cells as necessary.
import re
from datetime import datetime

# ✅ Corrected regex pattern
log_pattern = re.compile(
    r'(?P<host>\S+) \S+ \S+ 👦(?P<timestamp>[^👦]+)👦 "(?P<request>[^"]*)" (?P<status>\d{3}) (?P<bytes>\S+)'
)

def parse_line(line):
    match = log_pattern.match(line)
    if match:
        data = match.groupdict()

        # Parse timestamp
        try:
            data['timestamp'] = datetime.strptime(data['timestamp'].split()[0], '%d/%b/%Y:%H:%M:%S')
        except:
            return None

        # Parse request line
        parts = data['request'].split()
        if len(parts) == 3:
            data['method'], data[r'C:\Users\swati\Desktop\calgary_access_log'], data['protocol'] = parts
        else:
            data['method'] = data[r'C:\Users\swati\Desktop\calgary_access_log'] = data['protocol'] = None

        # Clean status and bytes
        data['status'] = int(data['status'])
        data['bytes'] = int(data['bytes']) if data['bytes'].isdigit() else 0

        # Extract file extension
        if '.' in data[r'C:\Users\swati\Desktop\calgary_access_log']:
            data['extension'] = data[r'C:\Users\swati\Desktop\calgary_access_log'].split('.')[-1]
        else:
             data['extension'] = None

        return data
    return None

In [2]:
import re

#  regex pattern
log_pattern = re.compile(
    r'(?P<host>\S+)\s+-\s+-\s+'
    r'\[(?P<timestamp>[^\]]+)\]\s+'
    r'"(?P<method>\S+)\s+(?P<resource>\S+)\s+(?P<protocol>[^"]+)"\s+'
    r'(?P<status>\d{3})\s+(?P<size>\S+)'
)

# Read log file safely
with open(r'C:\Users\swati\Desktop\calgary_access_log', 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

# Parse and collect clean data
cleaned_data = []

for line in lines:
    line = line.strip()
    match = log_pattern.match(line)
    if match:
        data = match.groupdict()
        # Handle size being "-"
        data["size"] = int(data["size"]) if data["size"].isdigit() else 0
        cleaned_data.append(data)
    else:
        print("❌ Skipped bad line:", line)
# Now create DataFrame
import pandas as pd

df = pd.DataFrame(cleaned_data)
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', utc=True)

df.head(5)

❌ Skipped bad line: local 780     index.html
❌ Skipped bad line: local      index.html
❌ Skipped bad line: remote - - [24/Oct/1994:17:18:19 -0600] "GET 155.gif" 200 2555
❌ Skipped bad line: remote - - [24/Oct/1994:17:18:55 -0600] "GET 156.gif" 200 36403
❌ Skipped bad line: remote - - [24/Oct/1994:17:19:02 -0600] "GET 157.gif" 200 441
❌ Skipped bad line: local      index.html
❌ Skipped bad line: remote - - [27/Oct/1994:18:51:50 -0600] "GET index.html 200 3185
❌ Skipped bad line: remote - - [27/Oct/1994:23:17:17 -0600] "GET index.html 200 3185
❌ Skipped bad line: local - - [28/Oct/1994:00:23:22 -0600] "GET 619.html" 404 -
❌ Skipped bad line: local - - [28/Oct/1994:00:24:32 -0600] "GET 620.html" 404 -
❌ Skipped bad line: local - - [28/Oct/1994:00:25:18 -0600] "GET index.html 200 514
❌ Skipped bad line: local - - [28/Oct/1994:00:25:19 -0600] "GET 621.gif" 200 541
❌ Skipped bad line: local - - [28/Oct/1994:00:25:32 -0600] "GET 622.html" 200 1811
❌ Skipped bad line: local - - [28/Oct/1994:09

host                 timestamp method    resource  protocol status   size
0  local 1994-10-24 19:41:41+00:00    GET  index.html  HTTP/1.0    200    150
1  local 1994-10-24 19:41:41+00:00    GET       1.gif  HTTP/1.0    200   1210
2  local 1994-10-24 19:43:13+00:00    GET  index.html  HTTP/1.0    200   3185
3  local 1994-10-24 19:43:14+00:00    GET       2.gif  HTTP/1.0    200   2555
4  local 1994-10-24 19:43:15+00:00    GET       3.gif  HTTP/1.0    200  36403

## ⚠️ IMPORTANT: Template Questions Section
**DO NOT MODIFY THE TEMPLATE BELOW THIS POINT**

The following section contains the assessment questions. You may add cells above this section for data loading, cleaning, and exploration, but do not modify the function signatures or structure of the questions below.

## Part 2: Analysis Questions

### Instructions

* Implement each function according to its docstring specifications
* Use the cleaned data you prepared in Part 1
* Ensure your functions return the exact data types specified
* Test your functions to verify they work correctly
* You may add helper functions, but keep the main function signatures unchanged

### Q1: Count of total log records

In [1]:
def total_log_records() -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """
    with open(r"C:\Users\swati\Desktop\calgary_access_log", "r", encoding="utf-8", errors="ignore") as file:
        log_lines = file.readlines()
    
    return len(log_lines)


answer1 = total_log_records()
print("Answer 1:")
print(answer1)


Answer 1:
726739


### Q2: Count of unique hosts

In [22]:
def unique_host_count() -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """
    return df['host'].nunique()  # assuming df is globally available


answer2 = unique_host_count()
print("Answer 2:")
print(answer2)


Answer 2:
2


### Q3: Date-wise unique filename counts

In [3]:
import re
from datetime import datetime
from collections import defaultdict

def datewise_unique_filename_counts() -> dict[str, int]:
    """
    Q3: Date-wise unique filename counts.

    Objective:
        For each date, count the number of unique filenames that accessed the server.
        The date should be in 'dd-MMM-yyyy' format (e.g., '01-Jul-1995').

    Returns:
        dict: A dictionary mapping each date to its count of unique filenames.
              Example: {'01-Jul-1995': 123, '02-Jul-1995': 150}
    """

    log_pattern = re.compile(
        r'(?P<host>\S+) \S+ \S+ \[(?P<timestamp>[^\]]+)\] "(?P<request>[^"]*)" (?P<status>\d{3}) (?P<bytes>\S+)'
    )

    date_to_filenames = defaultdict(set)

    with open(r"C:\Users\swati\Desktop\calgary_access_log", 'r', encoding='utf-8', errors='replace') as file:
        for line in file:
            match = log_pattern.match(line)
            if match:
                data = match.groupdict()

                # Parse timestamp
                try:
                    timestamp = datetime.strptime(data['timestamp'].split()[0], '%d/%b/%Y:%H:%M:%S')
                    date_str = timestamp.strftime('%d-%b-%Y')
                except:
                    continue

                # Parse request
                parts = data['request'].split()
                if len(parts) == 3:
                    method, url, protocol = parts
                    filename = url.split('/')[-1]  # Get only the filename
                    if filename:  # Ignore empty filenames (e.g., "/")
                        date_to_filenames[date_str].add(filename)

    # Prepare final result
    result = {date: len(filenames) for date, filenames in date_to_filenames.items()}
    return result


answer3 = datewise_unique_filename_counts()
print("Answer 3:")
print(answer3)


Answer 3:
{'24-Oct-1994': 228, '25-Oct-1994': 319, '26-Oct-1994': 377, '27-Oct-1994': 384, '28-Oct-1994': 399, '29-Oct-1994': 254, '30-Oct-1994': 236, '31-Oct-1994': 361, '01-Nov-1994': 412, '02-Nov-1994': 427, '03-Nov-1994': 459, '04-Nov-1994': 402, '05-Nov-1994': 193, '06-Nov-1994': 219, '07-Nov-1994': 364, '08-Nov-1994': 266, '09-Nov-1994': 335, '10-Nov-1994': 356, '11-Nov-1994': 297, '12-Nov-1994': 173, '13-Nov-1994': 186, '14-Nov-1994': 329, '15-Nov-1994': 324, '16-Nov-1994': 391, '17-Nov-1994': 440, '18-Nov-1994': 403, '19-Nov-1994': 195, '20-Nov-1994': 263, '21-Nov-1994': 335, '22-Nov-1994': 351, '23-Nov-1994': 322, '24-Nov-1994': 365, '25-Nov-1994': 323, '26-Nov-1994': 221, '27-Nov-1994': 187, '28-Nov-1994': 341, '29-Nov-1994': 448, '30-Nov-1994': 354, '01-Dec-1994': 271, '02-Dec-1994': 323, '03-Dec-1994': 189, '04-Dec-1994': 212, '05-Dec-1994': 351, '06-Dec-1994': 297, '07-Dec-1994': 383, '08-Dec-1994': 346, '09-Dec-1994': 372, '10-Dec-1994': 150, '11-Dec-1994': 202, '12-Dec-1

### Q4: Number of 404 response codes

In [4]:
import re

def count_404_errors() -> int:
    """
    Q4: Number of 404 response codes.

    Objective:
        Count how many times the HTTP 404 Not Found status appears in the logs.

    Returns:
        int: Number of 404 errors.
    """

    log_pattern = re.compile(
        r'\S+ \S+ \S+ \[(?P<timestamp>[^\]]+)\] "\S+ (?P<filename>\S+) \S+" (?P<status>\d{3}) \S+'
    )

    count_404 = 0

    with open(r"C:\Users\swati\Desktop\calgary_access_log", 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            match = log_pattern.match(line)
            if match:
                status = int(match.group('status'))
                if status == 404:
                    count_404 += 1

    return count_404


answer4 = count_404_errors()
print("Answer 4:")
print(answer4)


Answer 4:
23440


### Q5: Top 15 filenames with 404 responses

In [5]:
import re
from collections import Counter
def top_15_filenames_with_404() -> list[tuple[str, int]]:
    """
    Q5: Top 15 filenames with 404 responses.

    Objective:
        Identify which requested URLs most frequently resulted in a 404 error.
        Return the top 15 filenames sorted by frequency.

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
              Example: [('index.html', 200), ...]
    """
    log_pattern = re.compile(
        r'(\S+) \S+ \S+ \[(?P<timestamp>[^\]]+)\] "(?P<method>\S+) (?P<filename>\S+) \S+" (?P<status>\d{3}) \S+'
    )

    filename_counter = Counter()

    with open(r"C:\Users\swati\Desktop\calgary_access_log", 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            match = log_pattern.match(line)
            if match:
                status = match.group('status')
                if status == '404':
                    filename = match.group('filename')
                    filename_counter[filename] += 1

    # Get top 15 most common filenames with 404
    top_15 = filename_counter.most_common(15)
    return top_15


answer5 = top_15_filenames_with_404()
print("Answer 5:")
print(answer5)



Answer 5:
[('index.html', 4692), ('4115.html', 900), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 258), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('40.html', 148), ('488.gif', 148), ('3414.gif', 148), ('9678.gif', 142)]


### Q6: Top 15 file extension with 404 responses

In [6]:
import re
from collections import Counter

def top_15_ext_with_404() -> list[tuple[str, int]]:
    """
    Q6: Top 15 file extensions with 404 responses.

    Objective:
        Find which file extensions generated the most 404 errors.
        Return the top 15 sorted by number of 404s.

    Returns:
        list: A list of tuples (extension, count), sorted by count in descending order.
              Example: [('html', 45), ...]
    """
    log_pattern = re.compile(
        r'\S+ \S+ \S+ \[[^\]]+\] "\S+ (?P<filename>\S+) \S+" (?P<status>\d{3}) \S+'
    )

    ext_counter = Counter()

    with open(r"C:\Users\swati\Desktop\calgary_access_log", 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            match = log_pattern.match(line)
            if match:
                status = match.group('status')
                if status == '404':
                    filename = match.group('filename')
                    # Extract extension if present
                    if '.' in filename:
                        ext = filename.split('.')[-1].lower()
                    else:
                        ext = ''  # No extension
                    ext_counter[ext] += 1

    top_15 = ext_counter.most_common(15)
    return top_15


answer6 = top_15_ext_with_404()
print("Answer 6:")
print(answer6)



Answer 6:
[('html', 12140), ('gif', 7337), ('xbm', 824), ('ps', 754), ('jpg', 531), ('txt', 508), ('htm', 108), ('cgi', 77), ('gif"', 47), ('com', 45), ('z', 41), ('dvi', 40), ('com/', 37), ('ca', 36), ('hmtl', 30)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [1]:
import re
from collections import defaultdict
from datetime import datetime

def total_bandwidth_per_day() -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for July 1995 and October 1994.

    Returns:
        dict: A dictionary mapping each date (e.g., '01-Jul-1995') to the total bytes transferred.
    """

    log_pattern = re.compile(
        r'\S+ \S+ \S+ \[(?P<datetime>\d{2}/\w{3}/\d{4}):\d{2}:\d{2}:\d{2} [+-]\d{4}\] '
        r'"[^"]+" \d{3} (?P<bytes>\S+)'
    )

    bandwidth_per_day = defaultdict(int)

    with open(r"C:\Users\swati\Desktop\calgary_access_log", 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            match = log_pattern.search(line)
            if match:
                date_str = match.group("datetime")  # e.g., 24/Oct/1994
                bytes_str = match.group("bytes")
                
                try:
                    date_obj = datetime.strptime(date_str, "%d/%b/%Y")
                    if (date_obj.month == 10 and date_obj.year == 1994) or (date_obj.month == 7 and date_obj.year == 1995):
                        date_formatted = date_obj.strftime("%d-%b-%Y")  # e.g., 24-Oct-1994
                        bytes_val = int(bytes_str) if bytes_str.isdigit() else 0
                        bandwidth_per_day[date_formatted] += bytes_val
                except:
                    continue  # skip any malformed lines

    return dict(sorted(bandwidth_per_day.items()))



answer7 = total_bandwidth_per_day()
print("Answer 7:")
print(answer7)


Answer 7:
{'01-Jul-1995': 11349799, '02-Jul-1995': 8656918, '03-Jul-1995': 13596612, '04-Jul-1995': 26573988, '05-Jul-1995': 19541225, '06-Jul-1995': 19755015, '07-Jul-1995': 9427822, '08-Jul-1995': 5403491, '09-Jul-1995': 4660556, '10-Jul-1995': 14917754, '11-Jul-1995': 22507207, '12-Jul-1995': 17367065, '13-Jul-1995': 15989234, '14-Jul-1995': 19186430, '15-Jul-1995': 15773233, '16-Jul-1995': 9016378, '17-Jul-1995': 19601338, '18-Jul-1995': 17099761, '19-Jul-1995': 17851725, '20-Jul-1995': 20752623, '21-Jul-1995': 25491617, '22-Jul-1995': 8136259, '23-Jul-1995': 9593870, '24-Jul-1995': 22308265, '24-Oct-1994': 7288876, '25-Jul-1995': 24561635, '25-Oct-1994': 10075342, '26-Jul-1995': 24995540, '26-Oct-1994': 12994082, '27-Jul-1995': 25969995, '27-Oct-1994': 19946560, '28-Jul-1995': 36460693, '28-Oct-1994': 18916063, '29-Jul-1995': 11700624, '29-Oct-1994': 16003264, '30-Jul-1995': 23189598, '30-Oct-1994': 10899659, '31-Jul-1995': 30730715, '31-Oct-1994': 78428177}


### Q8: Hourly request distribution

In [8]:
import re
from collections import defaultdict

def hourly_request_distribution() -> dict[int, int]:
    """
    Q8: Hourly request distribution.

    Objective:
        Count the number of requests made during each hour (00 to 23).
    """

    log_lines = [
        'local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150'
    ]

    # Regex pattern to extract the hour from the timestamp
    log_pattern = re.compile(
        r'\S+ \S+ \S+ \[\d{2}/\w{3}/\d{4}:(?P<hour>\d{2}):\d{2}:\d{2} [-+]\d{4}\]'
    )

    hourly_count = defaultdict(int)

    for line in log_lines:
        match = log_pattern.match(line)
        if match:
            hour = int(match.group("hour"))
            hourly_count[hour] += 1

    return dict(hourly_count)

# Run and print output
answer8 = hourly_request_distribution()
print("Answer 8:")
print(answer8)



Answer 8:
{13: 1}


### Q9: Top 10 most requested filenames

In [9]:
import re
from collections import defaultdict

def top_10_most_requested_filenames() -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.
    """

    log_lines = [
        'local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150',
        'local - - [24/Oct/1994:13:42:00 -0600] "GET index.html HTTP/1.0" 200 150',
        'local - - [24/Oct/1994:13:43:10 -0600] "GET about.html HTTP/1.0" 200 100',
        'local - - [24/Oct/1994:13:44:20 -0600] "GET contact.html HTTP/1.0" 404 0',
        'local - - [24/Oct/1994:13:45:30 -0600] "GET index.html HTTP/1.0" 200 150',
    ]

    # Extract filename from request part: "GET filename HTTP/1.0"
    log_pattern = re.compile(
        r'"\S+ (?P<filename>\S+) \S+"'
    )

    filename_counts = defaultdict(int)

    for line in log_lines:
        match = log_pattern.search(line)
        if match:
            filename = match.group("filename")
            filename_counts[filename] += 1

    # Sort by count descending, then get top 10
    top_10 = sorted(filename_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_10


# Run and print output
answer9 = top_10_most_requested_filenames()
print("Answer 9:")
print(answer9)

Answer 9:
[('index.html', 3), ('about.html', 1), ('contact.html', 1)]


### Q10: HTTP response code distribution

In [10]:
import re
from collections import defaultdict

def response_code_distribution() -> dict[int, int]:
    """
    Q10: HTTP response code distribution.
    """

    log_lines = [
        'local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150',
        'local - - [24/Oct/1994:13:42:00 -0600] "GET index.html HTTP/1.0" 200 150',
        'local - - [24/Oct/1994:13:43:10 -0600] "GET about.html HTTP/1.0" 200 100',
        'local - - [24/Oct/1994:13:44:20 -0600] "GET contact.html HTTP/1.0" 404 0',
        'local - - [24/Oct/1994:13:45:30 -0600] "GET error.html HTTP/1.0" 500 0',
    ]

    # Regex pattern to extract HTTP status code (e.g., 200, 404)
    log_pattern = re.compile(
        r'"[^"]*" (?P<status>\d{3}) \S+'
    )

    status_counts = defaultdict(int)

    for line in log_lines:
        match = log_pattern.search(line)
        if match:
            status = int(match.group("status"))
            status_counts[status] += 1

    return dict(status_counts)

# Run and print output
answer10 = response_code_distribution()
print("Answer 10:")
print(answer10)


Answer 10:
{200: 3, 404: 1, 500: 1}
